In [ ]:
#generar tabla Calendario
import pandas as pd
import locale

# Establecer el idioma en español
import locale

for loc in ['Spanish_Spain.1252', 'es_ES.UTF-8']:
    try:
        locale.setlocale(locale.LC_TIME, loc)
        break
    except locale.Error:
        continue
else:
    print("⚠️ No se pudo establecer el idioma español. Verifica la configuración regional de tu sistema.")

# Crear el rango de fechas
fechas = pd.date_range(start="2003-01-01", end="2025-12-01", freq='D')

# Crear DataFrame base
calendario = pd.DataFrame({'FECHA_DATETIME': fechas})

# Formato de fecha como d/m/yyyy
calendario['FECHA'] = calendario['FECHA_DATETIME'].dt.strftime('%#d/%#m/%Y')

# Año y mes
calendario['AÑO'] = calendario['FECHA_DATETIME'].dt.year
calendario['MES_NUM'] = calendario['FECHA_DATETIME'].dt.month
calendario['MES_NOMBRE'] = calendario['FECHA_DATETIME'].dt.strftime('%B')

# Trimestre (formato: "Trimestre 1", "Trimestre 2", etc.)
calendario['TRIMESTRE'] = calendario['FECHA_DATETIME'].dt.quarter.apply(lambda x: f'Trimestre {x}')

# Cuatrimestre (enero-abril = 1, mayo-agosto = 2, sep-dic = 3 → formato: "Cuatrimestre 1", etc.)
calendario['CUATRIMESTRE'] = ((calendario['MES_NUM'] - 1) // 4 + 1).apply(lambda x: f'Cuatrimestre {x}')

# Semana del año
calendario['NUMERO_SEMANA'] = calendario['FECHA_DATETIME'].dt.isocalendar().week

# Día de la semana
calendario['DIA_NOMBRE'] = calendario['FECHA_DATETIME'].dt.strftime('%A') 

#Lunes: 1, Martes:2 ....
calendario['DIA_NUM_SEMANA'] = calendario['FECHA_DATETIME'].dt.weekday + 1

# Día del año
calendario['DIA_DEL_AÑO'] = calendario['FECHA_DATETIME'].dt.dayofyear

# Eliminar la columna datetime si no se necesita
calendario = calendario.drop(columns=['FECHA_DATETIME'])

# Guardar como CSV
calendario.to_csv("Calendario.csv", index=False, encoding='utf-8')

In [8]:
#generar tabla pais
import polars as pl

dt = pl.read_csv('HURTO_A_VEH_CULOS_20250501.csv', encoding='utf8')

Departamentos_unicos = (
    dt.select(["DEPARTAMENTO", "MUNICIPIO"])
    .unique()
    .sort(["DEPARTAMENTO","MUNICIPIO"])
    .with_columns([
        pl.lit("COLOMBIA").alias("PAÍS")
    ])
)
Departamentos_unicos.write_csv("Tabla_pais.csv")